# Analyzing Relationships About Videos in a YouTube Channel

 > Team 34. 20200549 Junha Jang, 20220112 Sangoh Kim, 20220025 Yejune Ko



In [ ]:
channel_name = '우고 - 군대가기 싫다'
DATA_PATH = './drive/MyDrive/CS471_project/data'

# 0. Setup
1. This project **uses GPU**. Therefore, please change the runtime type to `T4 GPU`.
2. Unzip `CS471_project.zip` and place it right under `MyDrive`. After that, your Google drive's directory structure should looks like below. (If `processed/{channel_name}`, `raw/{channel_name}` directories not appear, then please manually add them in your Google drive)<br>
data download link : https://drive.google.com/file/d/1pCVjnBQfEdjcNg2jatTdSBZdy6OfPso5/view?usp=drive_link
```
# This is formatted as dir
├── MyDrive
│   ├── CS471_project
│   │   ├── data
│   │   │   ├── raw
│   │   │   │   ├──{Channel Name}.zip
│   │   │   ├── processed
│   │   │   │   ├──{Channel Name}  # empty dir
│   │   │   ├── final
│   │   │   │   ├──{Channel Name}  # empty dir
```

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

# 1. Install required python packages via pip

In [ ]:
!pip install asttokens==2.4.1
!pip install colorama==0.4.6
!pip install decorator
!pip install exceptiongroup==1.2.1
!pip install executing==2.0.1
!pip install ipython
!pip install jedi==0.19.1
!pip install matplotlib-inline==0.1.7
!pip install numpy
!pip install pandas==2.0.3
!pip install parso==0.8.4
!pip install prompt-toolkit==3.0.43
!pip install pure-eval==0.2.2
!pip install Pygments==2.18.0
!pip install python-dateutil
!pip install pytz==2024.1
!pip install scikit-network==0.32.1
!pip install scipy==1.13.0
!pip install six==1.16.0
!pip install stack-data==0.6.3
!pip install tqdm==4.66.4
!pip install traitlets==5.14.3
!pip install typing_extensions==4.11.0
!pip install tzdata==2024.1
!pip install wcwidth==0.2.13
!pip install torch==2.3.0
!pip install matplotlib
!pip install torcheval==0.0.7
!pip install scikit-learn==1.5.0

In [ ]:
import csv
import os
import math
import copy
import datetime
import random
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from typing import Tuple, List
from tqdm import tqdm
from sknetwork.clustering import Louvain
from torcheval.metrics.functional import binary_f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

# 2. Generating Graph Dataset

The following code cell will unzip the raw YouTube Studio data. It will take about **15 minutes** (depends on network connection)

In [ ]:
zip_dir = f"/content/drive/My Drive/CS471_project/data/raw/{channel_name}.zip"
zip_ref = zipfile.ZipFile(zip_dir, 'r')

dir = "/content/drive/My Drive/CS471_project/data/raw/"
zip_ref.extractall(dir)
zip_ref.close()

Below `YTGraphGenerator` extracts nodes, edges information from above unzipped raw data and store those graph dataset under `data/processed` directory

In [ ]:
class YTGraphGenerator:
    def __init__(self, channel_name):
        self.channel_name = channel_name
        self.channel_path = DATA_PATH + '/raw/' + channel_name
        self.video_path = self.channel_path + '/videos'

        self.graph_node_id_set = set()
        self.graph_edge_dict = {}

        self.graph_node_info_dict = {}

        self.modified_graph_edge_dict = {}
        self.cnt = 0

        self.graph_edge_list = []

    def process_video_list(self):
        print("Step 2-1: process_video_list")

        # Get a list of all files and directories in the specified directory
        self.video_id_list = os.listdir(self.video_path)
        self.origin_video_id_list = copy.deepcopy(self.video_id_list)
        print("Number of video : ", len(self.video_id_list))
        print()

    def process_yt_related(self):
        print("Step 2-2: process_yt_related")

        for video_id in tqdm(self.video_id_list, desc="Processing"):
            if video_id == '.DS_Store' or len(video_id) == 0:
                continue
            # print("video id: ", video_id)
            self.graph_node_id_set.add(video_id)
            directory_path = self.video_path + '/' + video_id + '/VIDEO/표 데이터.csv'
            with open(directory_path, mode='r', encoding='utf-8') as file:
                # 증가한 구독자[48], 좋아요[27], 조회수[1], 노출수[52], 노출클릭율[51], 수익[42], 댓글[23], 평균 조회율[49]
                csv_reader = csv.reader(file)
                header = next(csv_reader)
                for row in csv_reader:
                    self.graph_node_info_dict[video_id] = [row[48], row[27], row[1], row[52], row[51], row[42], row[23], row[49]]
                    break

            directory_path = self.video_path + '/' + video_id + '/VIDEO/총계.csv'
            with open(directory_path, mode='r', encoding='utf-8') as file:
                csv_reader = csv.reader(file)
                header = next(csv_reader)
                for row in csv_reader:
                    self.graph_node_info_dict[video_id].append(row[0])
                    break

            # Open the CSV file
            directory_path = self.video_path + '/' + video_id + '/TRAFFIC_SOURCE_DETAIL__YT_RELATED/표 데이터.csv'
            related_information = []
            with open(directory_path, mode='r', encoding='utf-8') as file:
                # Create a CSV reader object
                csv_reader = csv.reader(file)

                # Iterate over each row in the CSV file
                for row in csv_reader:
                    if len(row) < 4:
                        continue
                    # Each row is a list where each element represents a column value
                    # print(row)
                    related_video_id = row[0][11:]
                    related_video_view = row[3]
                    related_information.append([video_id, related_video_id, related_video_view])
                    # print("related video id: ", related_video_id, "related video view: ", related_video_view)

                    if (not related_video_view == '조회수') and len(related_video_view) > 0 and len(related_video_id) > 0 and int(related_video_view) > 1 and (related_video_id in self.origin_video_id_list):
                        self.graph_node_id_set.add(related_video_id)
                        self.graph_edge_dict[(video_id, related_video_id)] = int(related_video_view)
                        # print("related video id : ", related_video_id)
                        # print("len related vid id : ", len(related_video_id))
                        # print("related video view : ", related_video_view)


        csv_file_path = DATA_PATH + '/processed/' + self.channel_name + '/node_information.csv'
        with open(csv_file_path, 'w', newline='') as file:
            csv_writer = csv.writer(file)

            csv_writer.writerow(['video_id', 'subscriber', 'like', 'view', 'exposure', 'exposure_click_rate', 'revenue', 'comment', 'average_view_rate', 'uploaded_time'])
            for row in self.graph_node_info_dict.items():
                line = [row[0]] + row[1]
                csv_writer.writerow(line)

        print("Number of nodes : ", len(self.graph_node_id_set))
        print("Number of edges : ", len(self.graph_edge_dict))
        print()

    def integrate_edges(self):
        print("Step 2-3: integrate_edges")

        # integrate in-degree and out-degree
        for edge in self.graph_edge_dict:
            to_node_id = edge[0]
            from_node_id = edge[1]
            weight1 = self.graph_edge_dict[edge]
            weight2 = 0
            # print("to_node_id: ", to_node_id, "from_node_id: ", from_node_id)
            if (from_node_id, to_node_id) in self.graph_edge_dict:
                self.cnt += 1
                # print("from_node_id: ", from_node_id, "to_node_id: ", to_node_id)
                weight2 = self.graph_edge_dict[(from_node_id, to_node_id)]
                # del graph_edge_set[(from_node_id, to_node_id)]

            new_weight = (math.log(weight1 + 1) + math.log(weight2 + 1)) / 2
            self.modified_graph_edge_dict[(to_node_id, from_node_id)] = new_weight

            # del graph_edge_set[edge]

        print("Number of edges originally had edges in both direction  : ", self.cnt)
        print()

    def process_yt_related2(self):
        # Data for EvolveGCN
        print("Step 2-4: process_yt_related2")

        video_dict = {}
        for video_idx, video_id in enumerate(self.video_id_list):
            video_dict[video_id] = video_idx + 1

        for video_id in tqdm(self.video_id_list, desc="Processing"):
            if video_id == '.DS_Store' or len(video_id) == 0:
                continue
            # print("video id: ", video_id)
            # self.graph_node_id_set.add(video_id)
            directory_path = self.video_path + '/' + video_id + '/TRAFFIC_SOURCE_DETAIL__YT_RELATED/차트 데이터.csv'
            with open(directory_path, mode='r', encoding='utf-8') as file:
                csv_reader = csv.reader(file)
                header = next(csv_reader)

                prev_source = prev_target = None
                for row in csv_reader:
                    source = video_dict.get(video_id, 0)
                    target = video_dict.get(row[1].split("YT_RELATED.")[-1], 0)
                    if source == 0 or target == 0:
                        continue

                    if prev_source != source or prev_target != target:
                        weight = 0
                    weight += int(row[4])
                    time = row[0]

                    # if weight > 0:
                    self.graph_edge_list.append([source, target, weight, time])

                    prev_source = source
                    prev_target = target

        def parse_date(date_str):
            return datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

        def calculate_date_diff(older_date, newer_date):
            return (newer_date - older_date).days

        self.graph_edge_list.sort(key=lambda x: parse_date(x[3]))
        oldest_date = parse_date(self.graph_edge_list[0][3])

        for row in self.graph_edge_list:
            current_date = parse_date(row[3])
            date_diff = calculate_date_diff(oldest_date, current_date)
            row[3] = date_diff

        # print(self.graph_edge_list[-1][3])

        csv_file_path = DATA_PATH + '/processed/' + self.channel_name + '/egcn.csv'
        with open(csv_file_path, 'w', newline='') as file:
            csv_writer = csv.writer(file)

            csv_writer.writerow(['source', 'target', 'weight', 'time'])
            for row in self.graph_edge_list:
                csv_writer.writerow(row)


    def write_graph(self):
        print("Step 2-4: write_graph")

        # download nodes, edges
        # dir = '/content/drive/MyDrive/CS471_project/Example_YT_Dataset/nodes.csv'
        dir = DATA_PATH + '/processed/' + self.channel_name + '/nodes.csv'

        with open(dir, 'w', newline='') as file:
            # Create a CSV writer object
            csv_writer = csv.writer(file)

            # Write each row of data to the CSV file
            for node in self.graph_node_id_set:
                csv_writer.writerow([node])

        # dir = '/content/drive/MyDrive/CS471_project/Example_YT_Dataset/edges.csv'
        dir = DATA_PATH + '/processed/' + self.channel_name + '/edges.csv'

        with open(dir, 'w', newline='') as file:
            # Create a CSV writer object
            csv_writer = csv.writer(file)

            # Write each row of data to the CSV file
            for edge in self.modified_graph_edge_dict:
                line = [edge[0], edge[1], self.modified_graph_edge_dict[edge]]
                csv_writer.writerow(line)
        print()

In [ ]:
step2 = YTGraphGenerator(channel_name)
step2.process_video_list()
step2.process_yt_related()
step2.integrate_edges()
step2.write_graph()

# 3. Analyzing the Graph with Louvain Method
The code in this section applies Louvain Method to our graph dataset, compute modularity, and store the clustering result.

In [ ]:
# compute_normalized_cut: louvain method를 통해 clustering 된 그래프의 normalized cut 값을 계산해보기 위해서 정의
def compute_normalized_cut(adj: sp.spmatrix,  # spmatrix: scipy.sparse의 모든 sparse matrix의 super class
                           labels: np.ndarray,
                           num_communities: int) -> Tuple[np.ndarray, np.float_]:
    adj_coo = adj.tocoo()
    edgeList = list(zip(adj_coo.row, adj_coo.col, adj_coo.data))  # edge list 만들 때 coo_matrix 형태가 유용

    cut = np.zeros(num_communities)
    vol = np.zeros(num_communities)

    for (u, v, w) in edgeList:
        # 서로 연결된 모든 조합에 대해서 deg 정보 계산.
        # 만약 속하는 cluster가 다르다면 cut 정보 계산.
        vol[labels[u]] += w  # deg(cluster of u)를 계산하는 과정
        if labels[u] != labels[v]:
            cut[labels[u]] += w

    normalized_cuts = cut / vol

    return normalized_cuts, np.sum(normalized_cuts)

# clustering evaluation을 위한 modularity 계산 함수
def compute_modularity(adj: sp.spmatrix,
                       labels: np.ndarray) -> float:
    adj_coo = adj.tocoo()
    edgeList = list(zip(adj_coo.row, adj_coo.col, adj_coo.data))

    edgeList_np = np.array(edgeList)

    modularity = 0.0
    m_tot = adj.sum()

    for (u, v, w) in edgeList:
        if labels[u] == labels[v]:
            k_u = edgeList_np[edgeList_np[:, 0] == u][:, 2].sum()
            k_v = edgeList_np[edgeList_np[:, 0] == v][:, 2].sum()
            modularity += adj[u, v] - k_u * k_v / m_tot

    return modularity / m_tot

In [ ]:
class YTClustering:
    def __init__(self, channel_name):
        print("Step 3-1 : YTClustering")

        self.base_path = DATA_PATH + '/processed/' + channel_name

        self.nodes_str = pd.read_csv(f'{self.base_path}/node_information.csv')['video_id'].values.squeeze()

        self.edges_csv = open(f'{self.base_path}/edges.csv')
        self.edges_csv_reader = csv.reader(self.edges_csv)

        self.generator()
        self.applier()

        print()

    def generator(self):
        nodes_list = self.nodes_str.tolist()
        nodes_num = [i for i in range(len(nodes_list))]

        node_dict = dict(zip(nodes_list, nodes_num))

        num_to_node_dict = dict(zip(nodes_num, nodes_list))

        # edge 정보를 coo_matrix로 표현
        data, row, col = [], [], []

        for (src, dst, weight) in self.edges_csv_reader:
            src_num, dst_num = node_dict[src], node_dict[dst]
            data.append(float(weight))
            row.append(src_num)
            col.append(dst_num)
        data = np.array(data)
        row = np.array(row)
        col = np.array(col)

        print(f"# of nodes: {len(nodes_num)}")
        print(f"# of edges: {data.shape}")

        self.sparse_coo_matrix = sp.coo_matrix((data, (row, col)), shape=(len(nodes_num), len(nodes_num)))
        self.sparse_csr_matrix = sp.csr_matrix(self.sparse_coo_matrix)

    def applier(self):
        louvain = Louvain()
        labels = louvain.fit_predict(self.sparse_csr_matrix)

        unique_communities, counts = np.unique(labels, return_counts = True)
        num_communities = len(unique_communities)
        print(f"Number of total communities: {num_communities}")
        print(f"Number of nodes in each community: {counts}")

        normalized_cuts, NC = compute_normalized_cut(self.sparse_csr_matrix, labels, num_communities)
        Q = compute_modularity(self.sparse_csr_matrix, labels)

        print(f"Normalized cut of each cluster: {normalized_cuts}")
        print(f"Graph normalized cut: {NC:.4f}")
        print(f"Modularity: {Q:.4f}")

        with open(self.base_path + '/cluster.csv', 'w', newline='') as file:
            # Create a CSV writer object
            csv_writer = csv.writer(file)
            csv_writer.writerow(labels)  # nodes.csv의 node 순서와 동일하게 매핑됨.

In [ ]:
step3 = YTClustering(channel_name)

# 4. Analyzing Clustered Data
The cluster labels obtained through the Louvain method were used to summarize the characteristics of each cluster based on studio data.


We have created csv report for all labels in the studio. It will be created in `/data/final/{channel_name}`. Since there are too many labels, we have selected 8 main features `'increased subscriber', 'like', 'view', 'exposure', 'exposure_click_rate', 'revenue', 'comment', 'average_view_rate', 'uploaded_time'`. Result about the 8 features is summarized in `/data/final/{channel_name}/VIDEO.csv`.

`DIMESIONS` are the features that are related to each video. Select the features from the following list and add them to the code.<br>
(`DAY` feature takes more than an hour on colab)

```
DIMENSIONS = [
    'VIDEO', 'PLAYER_APP_TYPE', 'CAPTION_LANGUAGE', 'DEVICE_OS_TYPE', 'VIEWER_GENDER', 'INFO_CARD_TYPE',
    'VIEWER_AGE', 'SHARING_SERVICE', 'DAY', 'CITY', 'PLAYBACK_LOCATION_TYPE',
    'IS_CROSS_LANGUAGE', 'EMBEDDED_PLAYER_MODE', 'ADTYPES', 'EARNINGS_SOURCE_ALL', 'SUBSCRIBED_TO_UPLOADER_STATE',
    'LOYALTY_STATE', 'VIDEO_METADATA_LANGUAGE', 'TRAFFIC_SOURCE_TYPE', 'TRANSACTION_BUSINESS_MODEL',
    'SUBSCRIPTION_SOURCE_TYPE', 'DEVICE_PLATFORM_TYPE', 'ENDSCREEN_ELEMENT_TYPE', 'ENDSCREEN_ELEMENT_ID',
    'INFO_CARD_ID', 'CREATOR_CONTENT_TYPE', 'POST', 'COUNTRY'
]
```

In [ ]:
DIMENSIONS = [
    'VIDEO', 'VIEWER_GENDER', 'VIEWER_AGE', 'ADTYPES',
    'EARNINGS_SOURCE_ALL', 'SUBSCRIBED_TO_UPLOADER_STATE',
    'SUBSCRIPTION_SOURCE_TYPE', 'DEVICE_PLATFORM_TYPE', 'CREATOR_CONTENT_TYPE'
]

In [ ]:
class YTGraphAnalyzer:
    def __init__(self, channel_name):
        self.processed_dimension = []

        print("Step 4-1 : process_dimension")
        self.process_dimension()
        print()

        print("Step 4-2 : dataframe_generator")
        for dimension in tqdm(DIMENSIONS, desc="Processing"):
            self.dataframe_generator(channel_name, dimension)
        print()

        print("Step 4-3 : video_dataframe_processor")
        self.video_dataframe_processor(channel_name)
        print()

    def process_dimension(self):
        # 기본 경로 설정
        base_directory_path = DATA_PATH + '/raw/' + channel_name + '/videos'

        # 각 DIMENSION의 최대 항목 수를 저장할 딕셔너리
        max_items_per_dimension = {dimension: 0 for dimension in DIMENSIONS}
        min_items_per_dimension = {dimension: 3 for dimension in DIMENSIONS}

        # base_directory_path 경로 내의 모든 폴더를 순회
        for item in tqdm(os.listdir(base_directory_path), desc="Processing"):
            item_path = os.path.join(base_directory_path, item)
            # print(item_path)
            # 폴더인지 확인
            if os.path.isdir(item_path):
                # DIMENSIONS 리스트 내의 각 폴더에 대해
                for dimension in DIMENSIONS:
                    dimension_path = os.path.join(item_path, dimension)
                    # 폴더 내 항목 수 계산
                    # 해당 경로가 실제로 존재하는 폴더인지 확인
                    item_count = len(os.listdir(dimension_path)) if os.path.isdir(dimension_path) else 0

                    # 현재 항목 수가 이전 최댓값보다 큰 경우 업데이트
                    if item_count > max_items_per_dimension[dimension]:
                        max_items_per_dimension[dimension] = item_count
                    if item_count < min_items_per_dimension[dimension]:
                        min_items_per_dimension[dimension] = item_count

        # max와 min이 같은 dimension들을 찾아서 출력
        for dimension in DIMENSIONS:
            if max_items_per_dimension[dimension] == min_items_per_dimension[dimension]:
                self.processed_dimension.append(dimension)
                # print(f"{dimension}: {max_items_per_dimension[dimension]}")

        print(self.processed_dimension)

    def dataframe_generator(self, channel_name, dimension, type=None):
        # 기본 경로 설정
        videos_directory_path = os.path.join(DATA_PATH, 'raw', channel_name, 'videos')
        cluster_directory_path = os.path.join(DATA_PATH, 'processed', channel_name, 'cluster.csv')

        # clusters.csv에서 클러스터 정보 로드
        clusters = pd.read_csv(cluster_directory_path, header=None).squeeze()  # Series로 변환

        video_id_list = os.listdir(videos_directory_path)

        # 클러스터별 데이터 프레임을 저장할 딕셔너리
        cluster_data_frames = {}

        for video_id, cluster in zip(video_id_list, clusters):
            # print(video_id, cluster)

            video_directory_path = os.path.join(videos_directory_path, video_id, dimension, '표 데이터.csv')
            if os.path.exists(video_directory_path):
                # CSV 파일 로드
                df = pd.read_csv(video_directory_path, header=None)  # 헤더 없이 로드

                # 해당 클러스터의 리스트에 데이터 프레임 추가
                if cluster not in cluster_data_frames:
                    cluster_data_frames[cluster] = [df]
                else:
                    cluster_data_frames[cluster].append(df)

        # 클러스터별로 평균값 계산 및 저장
        for cluster, data_frames in cluster_data_frames.items():
            if not data_frames:
                print(f"클러스터 {cluster}에 대한 데이터가 없습니다.")
                continue

            # 첫 번째 데이터 프레임을 기준으로 삼음
            base_df = data_frames[0].copy()

            # 나머지 데이터 프레임들과 비교하여 같은 위치의 셀 평균 계산
            for i in range(1, len(data_frames)):
                current_df = data_frames[i]
                for row in range(min(len(base_df), len(current_df))):
                    for col in range(min(len(base_df.columns), len(current_df.columns))):
                        base_value = pd.to_numeric(base_df.iloc[row, col], errors='coerce')
                        current_value = pd.to_numeric(current_df.iloc[row, col], errors='coerce')
                        if pd.notnull(current_value):
                            if pd.notnull(base_value):
                                base_df.iloc[row, col] = np.mean([base_value, current_value])
                            else:
                                base_df.iloc[row, col] = current_value

            # 평균값을 담은 데이터 프레임을 CSV로 저장
            output_path = os.path.join(DATA_PATH, 'final', channel_name, f'{dimension}{cluster}.csv')
            base_df.to_csv(output_path, index=False, header=False)
            # print(f"클러스터 {cluster}의 결과가 {output_path}에 저장되었습니다.")

    def video_dataframe_processor(self, channel_name, dimension="VIDEO", type=None):
        # 모든 데이터프레임을 아래로 합치기
        df_list = []
        for cluster_index in tqdm(range(3), desc="Processing"):
            df_directory_path = os.path.join(DATA_PATH, 'final', channel_name, f'{dimension}{cluster_index}.csv')
            df_list.append(pd.read_csv(df_directory_path, header=0))
        df = pd.concat(df_list, ignore_index=True)

        # 특정 행들만 선택
        df = df.loc[:, ['조회수', '추가된 댓글 수', '좋아요', '예상 수익 (KRW)', '구독자', '평균 조회율 (%)', '노출 클릭률 (%)', '노출수']]

        # 행 이름 변경
        # df.rename(index={
        #     '조회수': 'Views',
        #     '추가된 댓글 수': 'Comments Added',
        #     '좋아요': 'Likes',
        #     '예상 수익 (KRW)': 'Estimated Revenue (KRW)',
        #     '구독자': 'Subscribers',
        #     '평균 조회율 (%)': 'Average Watch Percentage (%)',
        #     '노출 클릭률 (%)': 'Impression Click-Through Rate (%)',
        #     '노출수': 'Impressions'
        # }, inplace=True)

        # 출력 경로 설정
        output_path = os.path.join(DATA_PATH, 'final', channel_name, f'{dimension}.csv')

        # 새로운 CSV 파일로 저장
        df.to_csv(output_path, index=True, header=True)

Below code cell will take about 2 minutes.

In [ ]:
step4 = YTGraphAnalyzer(channel_name)
step4.video_dataframe_processor(channel_name)

# 5. Applying GNNs for Link Prediction

In [ ]:
SEED = 42


def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        torch.mps.manual_seed(seed)

torch.backends.cudnn.benchmark = True  # CuDNN 라이브러리의 최적화된 값
# 재현가능한 결과가 나오도록 함. 성능의 손실이 약간 있을 수 있음
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)  # 재현가능한 결과가 나오도록 함.
# os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":16:8"


set_seed(SEED)

In [ ]:
def set_device() -> torch.device:
    # device 설정
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device('mps')
    elif torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


device = set_device()
if device == torch.device('mps'):
    FLOAT_TYPE = torch.float32
else:
    FLOAT_TYPE = torch.float64
print(f"Using {device}")

In [ ]:
base_path = f'{DATA_PATH}/processed'

In [ ]:
node_information_df = pd.read_csv(f'{base_path}/{channel_name}/node_information.csv')

nodes_str = node_information_df['video_id'].values.squeeze()

# node 별로 번호를 할당
nums = [i for i in range(len(nodes_str))]
node2num_dict = dict(zip(nodes_str, nums))

node_features = node_information_df.drop(['video_id', 'uploaded_time'], axis=1).values

std_scaler = StandardScaler()
node_features = std_scaler.fit_transform(node_features)
node_features = torch.tensor(node_features, dtype=FLOAT_TYPE)

In [ ]:
def get_unix_timestamp(ts: str) -> float:
    date = np.datetime64(ts, 's')
    timestamp = (date - np.datetime64('1970-01-01T00:00:00', 's')) / np.timedelta64(1, 's')

    return timestamp


edges_tot = []  # src_num, dst_num, weights, uploaded_unix_timestamp
with open(f'{base_path}/{channel_name}/edges.csv') as edges_f:
    rdr = csv.reader(edges_f)
    for src_str, dst_str, weight_str in rdr:
        src_str, dst_str, weight_str = src_str.strip(), dst_str.strip(), weight_str.strip()
        src_timestamp = get_unix_timestamp(node_information_df[node_information_df['video_id'] == src_str]['uploaded_time'].item())
        dst_timestamp = get_unix_timestamp(node_information_df[node_information_df['video_id'] == dst_str]['uploaded_time'].item())
        src_num, dst_num = node2num_dict[src_str], node2num_dict[dst_str]
        edges_tot.append([src_num, dst_num, float(weight_str.strip()), src_timestamp if src_timestamp > dst_timestamp else dst_timestamp])

edges_tot = np.array(edges_tot, dtype=np.float64)
edges_int = edges_tot[:, [0, 1, 3]].astype(np.int64)
edges_int = edges_int[np.lexsort([edges_int[:, 1], edges_int[:, 0]])]
edges_int = torch.tensor(edges_int, dtype=torch.int64)

In [ ]:
def get_degree(node_features, edges: torch.Tensor) -> torch.Tensor:
    adj = torch.zeros(node_features.shape[0], node_features.shape[0], dtype=FLOAT_TYPE)
    adj[edges[:, 0], edges[:, 1]] = 1.0

    degree = adj.sum(dim=1)
    degree[degree == 0] = 1e-6

    return degree

In [ ]:
def get_str_datetime(ts: int) -> str:
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')


def split_index(train_size: float = 0.6, valid_size: float = 0.2, test_size: float = 0.2) -> Tuple[torch.Tensor]:
    train_idx = int(edges_int.shape[0] * train_size)
    msg_idx = int(edges_int.shape[0] * train_size * 0.8)
    valid_idx = int(edges_int.shape[0] * (train_size + valid_size))
    edges_ts_asc = edges_int[:, 2].sort()[0]

    start = get_str_datetime(edges_ts_asc[0].item())
    msg_end_ts = edges_ts_asc[msg_idx].item()
    msg_end = get_str_datetime(msg_end_ts)
    train_end_ts = edges_ts_asc[train_idx].item()
    train_end = get_str_datetime(edges_ts_asc[train_idx].item())
    valid_end_ts = edges_ts_asc[valid_idx].item()
    valid_end = get_str_datetime(edges_ts_asc[valid_idx].item())
    test_end = get_str_datetime(edges_ts_asc[-1].item())

    train_indices_msg = torch.where(edges_int[:, 2] < msg_end_ts)[0]
    train_indices_pred = torch.where((edges_int[:, 2] >= msg_end_ts) & (edges_int[:, 2] < train_end_ts))[0]
    val_indices = torch.where((edges_int[:, 2] >= train_end_ts) & (
        edges_int[:, 2] < valid_end_ts))[0]
    test_indices = torch.where(edges_int[:, 2] >= valid_end_ts)[0]

    print(f'Training (message passing & aggregation). ({train_indices_msg.shape[0]} edges): {start} ~ {msg_end}')
    print(f'Training (prediction). ({train_indices_pred.shape[0]} edges): {msg_end} ~ {train_end}')
    print(f'Validation.   ({val_indices.shape[0]} edges): {train_end} ~ {valid_end}')
    print(f'Test.        ({test_indices.shape[0]} edges): {valid_end} ~ {test_end}')

    return train_indices_msg, train_indices_pred, val_indices, test_indices


train_indices_msg, train_indices_pred, val_indices, test_indices = split_index()

In [ ]:
class GraphSageLayer(nn.Module):
    def __init__(self, dim_in: int, dim_out: int, p: float, agg_type: str):
        super(GraphSageLayer, self).__init__()

        self.dim_in = dim_in
        self.dim_out = dim_out
        self.agg_type = agg_type
        self.device = device

        self.act = nn.ReLU()

        if self.agg_type == 'gcn':
            self.weight = nn.Linear(
                self.dim_in, self.dim_out, bias=False, dtype=FLOAT_TYPE)  # W_l
            self.bias = nn.Linear(self.dim_in, self.dim_out,
                                  bias=False, dtype=FLOAT_TYPE)  # B_l
        elif self.agg_type == 'mean':
            self.weight = nn.Linear(
                2 * self.dim_in, self.dim_out, bias=False, dtype=FLOAT_TYPE)  # W_l
        elif self.agg_type == 'maxpool':
            self.linear_pool = nn.Linear(
                # W_{pool}, b
                self.dim_in, self.dim_in, bias=True, dtype=FLOAT_TYPE)
            self.weight = nn.Linear(
                2 * self.dim_in, self.dim_out, bias=False, dtype=FLOAT_TYPE)  # W_l
        else:
            raise RuntimeError(f"Unknown aggregation type: {self.agg_type}")


    def forward(self, feat: torch.Tensor, edge: torch.Tensor, degree: torch.Tensor) -> torch.Tensor:
        if self.agg_type == 'gcn':
            # aggregation
            X = torch.zeros(feat.shape[0], feat.shape[1],
                            dtype=FLOAT_TYPE).to(device)
            X.index_add_(0, edge[:, 0], feat[edge[:, 1]])
            X /= degree.reshape(feat.shape[0], -1)
            # NN
            out = self.act(self.weight(X) + self.bias(feat))
            # normalization
            out = F.normalize(out, p=2.0, dim=1)
            return out
        elif self.agg_type == 'mean':
            # aggregation
            X = torch.zeros(feat.shape[0], feat.shape[1],
                            dtype=FLOAT_TYPE).to(device)
            X.index_add_(0, edge[:, 0], feat[edge[:, 1]])
            X /= degree.reshape(feat.shape[0], -1)
            # concatenating self transformation
            X = torch.cat([X, feat], dim=1).to(device)
            # NN
            out = self.act(self.weight(X))
            # normalization
            out = F.normalize(out, p=2.0, dim=1)
            return out
        elif self.agg_type == 'maxpool':
            # self.linear_pool의 bias=True
            pooled_feat = self.act(self.linear_pool(feat))
            X = torch.zeros(
                pooled_feat.shape[0], pooled_feat.shape[1], dtype=FLOAT_TYPE).to(device)
            # scatter_reduce와 index_add 차이 주의
            X.scatter_reduce_(
                0, edge[:, 0].reshape(-1, 1).repeat(1, feat.shape[1]), pooled_feat[edge[:, 1]], reduce='amax', include_self=False)
            # forwarded feat가 아닌 원래 feat과 cat해야함.
            X = torch.cat([X, feat], dim=1).to(device)
            out = self.act((self.weight(X)))
            out = F.normalize(out, p=2.0, dim=1)
            return out
        else:
            raise RuntimeError(f"Unknown aggregation type: {self.agg_type}")


class GraphSageLinkPred(nn.Module):
    def __init__(self, num_layers_gnn: int, num_layers_nn: int, dim_in: int, dim_hidden: int, dim_hidden_nn: int, dim_out: int, p: float, agg_type: str):
        super(GraphSageLinkPred, self).__init__()

        self.num_layers_gnn = num_layers_gnn
        self.num_layers_nn = num_layers_nn
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        self.dim_hidden_nn = dim_hidden_nn
        self.dim_out = dim_out
        self.agg_type = agg_type
        self.p = p

        self.layers = nn.ModuleList()
        for layer in range(num_layers_gnn):
            self.layers.append(GraphSageLayer(
                dim_in if layer == 0 else dim_hidden, dim_hidden, p, agg_type))

        self.emb_layer = nn.Linear(dim_hidden, dim_out, bias=True, dtype=FLOAT_TYPE)

        # node 의 순서는 보존이 되는 상황

        layers_tmp = []
        for layer in range(num_layers_nn):
            linear_layer = nn.Linear(2 * dim_out if layer == 0 else dim_hidden_nn, dim_hidden_nn, bias=True, dtype=FLOAT_TYPE)
            layers_tmp.append(linear_layer)
            layers_tmp.append(nn.BatchNorm1d(dim_hidden_nn, dtype=FLOAT_TYPE))
            layers_tmp.append(nn.Dropout(p=p, inplace=False))
            layers_tmp.append(nn.ReLU())

        self.layers_link_pred = nn.Sequential(*layers_tmp)

        self.final_encoder = nn.Linear(dim_hidden_nn, 1, bias=True, dtype=FLOAT_TYPE)


    def forward(self, feat: torch.Tensor, edges_tot: torch.Tensor, edges_train: torch.Tensor, degree: torch.Tensor) -> torch.Tensor:
        x_in = feat
        for layer in self.layers:
            x_out = layer(x_in, edges_train, degree).to(device)
            x_in = x_out

        embedding = self.emb_layer(x_out)

        concated = torch.cat((embedding[edges_tot[:, 0]], embedding[edges_tot[:, 1]]), dim=1)
        out = self.layers_link_pred(concated).to(device)

        final = self.final_encoder(out)
        return final, embedding


In [ ]:
class GATLayer(nn.Module):
    def __init__(self, dim_in: int, dim_out: int, activation: lambda x: x, dropout: float = 0.5, alpha: float = 0.2):
        super(GATLayer, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.dropout = dropout
        self.activation = activation
        self.alpha = alpha

        self.W = nn.Parameter(torch.empty((dim_in, dim_out), dtype=FLOAT_TYPE))
        nn.init.kaiming_uniform_(self.W.data)
        self.a = nn.Parameter(torch.empty((2 * dim_out, 1), dtype=FLOAT_TYPE))
        nn.init.kaiming_uniform_(self.a.data)

        self.leakyrelu = nn.LeakyReLU(negative_slope=self.alpha)

    def forward(self, feat: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        # message passing (각각의 embedding을 먼저 linear transformation)
        message_tensor = feat @ self.W

        # attention weight 구하기
        attention_src_tensor = message_tensor @ self.a[:self.dim_out, :]
        attention_dst_tensor = message_tensor @ self.a[self.dim_out:, :]
        attention_coef_tensor = self.leakyrelu(
            attention_src_tensor[edges[:, 0]] + attention_dst_tensor[edges[:, 1]])  # e_uv
        attention_coef_tensor = attention_coef_tensor - attention_coef_tensor.max() + 1e-6
        exp_sum = torch.zeros((feat.shape[0], 1), dtype=FLOAT_TYPE).to(device)
        exp_sum.index_add_(0, edges[:, 0], attention_coef_tensor.exp())
        exp_sum = exp_sum + 1e-6
        attention_weight_tensor = attention_coef_tensor.exp() / \
            exp_sum[edges[:, 0]]
        attention_weight_tensor = F.dropout(
            attention_weight_tensor, p=self.dropout, training=self.training, inplace=False)

        neighbors_tensor = attention_weight_tensor * \
            message_tensor[edges[:, 1]]
        weighted_sum_tensor = torch.zeros((feat.shape[0], self.dim_out), dtype=FLOAT_TYPE).to(
            device).index_add_(0, edges[:, 0], neighbors_tensor)
        out = self.activation(weighted_sum_tensor)

        return out


class GAT(nn.Module):
    '''
    1. message passing
    2. attention 구하기
    3. attention으로 weighted sum 하고 linear transformation
    '''

    def __init__(self, feat_dim: int, emb_dim: int, num_layers_nn: int, dim_hidden: int, dropout: float = 0.5, alpha: float = 0.2, num_heads: int = 8) -> None:
        super(GAT, self).__init__()
        self.p = dropout
        self.num_heads = num_heads

        self.attn_heads = nn.ModuleList(
            [GATLayer(feat_dim, emb_dim, nn.ReLU(), dropout, alpha) for _ in range(num_heads)])

        '''
        Multihead Aggregation
        - concatenation (current)
        - mean, maxpool 등 적용해 볼수도
        '''
        layers_tmp = []
        for i in range(num_layers_nn):
            linear_layer = nn.Linear(
                num_heads * emb_dim * 2 if i == 0 else dim_hidden, dim_hidden, bias=True, dtype=FLOAT_TYPE)
            layers_tmp.append(linear_layer)
            layers_tmp.append(nn.BatchNorm1d(dim_hidden, dtype=FLOAT_TYPE))
            layers_tmp.append(nn.Dropout(p=self.p, inplace=False))
            layers_tmp.append(nn.ReLU())
        self.link_pred_layers = nn.Sequential(*layers_tmp)
        self.final_encoder = nn.Linear(dim_hidden, 1, bias=True, dtype=FLOAT_TYPE)

    def forward(self, feat: torch.Tensor, edges_tot: torch.Tensor, edges_train: torch.Tensor, degree: int):
        feat = F.dropout(feat, p=0.5, training=self.training)
        embeds = [head(feat, edges_train) for head in self.attn_heads]
        res = torch.cat(embeds, dim=1).to(device)
        res = F.dropout(res, p=0.5, training=self.training).to(device)

        embeds_res = torch.tensor(
            [embed.tolist() for embed in embeds], dtype=FLOAT_TYPE).to(device)

        x_in = torch.cat([res[edges_tot[:, 0]], res[edges_tot[:, 1]]], dim=1).to(device)
        out = self.link_pred_layers(x_in)

        return self.final_encoder(out), embeds_res.mean(dim=0)

In [ ]:
node_features = node_features.to(device)
edges_int = edges_int.to(device)
train_indices_msg = train_indices_msg.to(device)
train_indices_pred = train_indices_pred.to(device)
val_indices = val_indices.to(device)

In [ ]:
def train(model: nn.Module, feat: torch.Tensor, edge: torch.Tensor,
          train_indices_msg: torch.Tensor, train_indices_pred: torch.Tensor, valid_indices: torch.Tensor, test_indices: torch.Tensor, lr: float, num_epochs: int, threshold: float):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    epoch_best = 0
    best_valid_f1 = 0
    list_valid_f1, list_loss = [], []

    with tqdm(range(num_epochs), unit="epoch", desc="Training") as pbar:
        pbar.clear()
        for epoch in pbar:

            model.train()

            optimizer.zero_grad()

            degree = get_degree(feat.cpu(), edge.cpu()[train_indices_msg.cpu()]).to(device)
            pred, _ = model.forward(feat, edge, edge[train_indices_msg], degree)  # [# of edges, 1] 을 반환
            loss = F.binary_cross_entropy_with_logits(pred[train_indices_pred] + 1e-7, torch.ones(train_indices_pred.shape[0], 1).to(device))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # to prevent gradient exploding
            optimizer.step()

            list_loss.append(loss.item())

            model.eval()
            with torch.no_grad():
                pred, _ = model(feat, edge, edge[train_indices_msg], degree)
                train_f1 = binary_f1_score(pred[train_indices_pred].squeeze().sigmoid(), torch.ones(train_indices_pred.shape[0]).to(device), threshold=threshold)

                degree = get_degree(feat, edge[torch.cat([train_indices_msg, train_indices_pred])]).to(device)
                pred, _ = model(feat, edge, edge[torch.cat([train_indices_msg, train_indices_pred]).to(device)], degree)
                valid_f1 = binary_f1_score(pred[valid_indices].squeeze().sigmoid(), torch.ones(valid_indices.shape[0]).to(device), threshold=threshold)

                list_valid_f1.append(valid_f1.item())

                if best_valid_f1 < valid_f1:
                    best_valid_f1 = valid_f1
                    epoch_best = epoch
                    torch.save(model, './best_model.pt')

                postfix_new = ", ".join([f"f1_train: {train_f1:.7f} f1_val: {valid_f1:.7f} (best: {best_valid_f1:.7f} - Epoch {epoch_best})",
                                         f"loss: {loss:.7f}"])
                pbar.set_postfix_str(postfix_new)

    print(f"Best Epoch: {epoch_best}")
    print(f"Best Validation F1-score: {best_valid_f1}")

    best_model = torch.load("./best_model.pt", map_location=device)
    best_model.eval()
    with torch.no_grad():
        degree = get_degree(feat, edge[torch.cat([train_indices_msg, train_indices_pred, valid_indices])]).to(device)
        pred, _ = model(feat, edge, edge[torch.cat([train_indices_msg, train_indices_pred, valid_indices]).to(device)], degree)
        test_f1 = binary_f1_score(pred[test_indices].squeeze().sigmoid(), torch.ones(test_indices.shape[0]).to(device), threshold=threshold)

    print(f"Test F1-score: {test_f1}")

    return list_loss, list_valid_f1, test_f1

In [ ]:
def visualize_train_history(num_epochs: int, list_train_loss: torch.Tensor, list_valid_f1: torch.Tensor, title: str):
    X = np.arange(1, num_epochs + 1)
    _, ax1 = plt.subplots()

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Train BCE Loss')
    line1 = ax1.plot(X, list_train_loss, color='blue', label='Train')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Valid F1-score')
    line2 = ax2.plot(X, list_valid_f1, color='orange', label='Valid')

    lines = line1 + line2
    labels = [line.get_label() for line in lines]

    ax1.legend(lines, labels, loc="upper left", bbox_to_anchor=(1.1, 1))

    plt.title(title)
    plt.show()

In [ ]:
hyperparameters_graphsage = {
    "num_layers": 2,
    "num_layers_nn": 3,
    "num_epochs": 100,
    "dim_in": 8,
    "dim_hidden": 8,
    "dim_hidden_nn": 16,
    "dim_out": 2,
    "lr": 0.001,
    "dropout": 0.5,
    "agg_type": "mean",
    "threshold": 0.65
}

hyperparameters_gat = {
    "num_layers_nn": 4,
    "num_epochs": 100,
    "dim_in": 8,
    "dim_hidden_nn": 32,
    "dim_out": 2,
    "lr": 0.001,
    "dropout": 0.5,
    "alpha": 0.2,
    "num_heads": 8,
    "threshold": 0.65
}

In [ ]:
def train_graphsage(agg_type: str):
    hyperparameters_graphsage['agg_type'] = agg_type

    model = GraphSageLinkPred(
        hyperparameters_graphsage['num_layers'],
        hyperparameters_graphsage['num_layers_nn'],
        hyperparameters_graphsage['dim_in'],
        hyperparameters_graphsage['dim_hidden'],
        hyperparameters_graphsage['dim_hidden_nn'],
        hyperparameters_graphsage['dim_out'],
        hyperparameters_graphsage['dropout'],
        hyperparameters_graphsage['agg_type']
    ).to(device)

    list_loss, list_valid_f1, _ = train(
        model, node_features, edges_int, train_indices_msg, train_indices_pred, val_indices, test_indices,
        hyperparameters_graphsage['lr'], hyperparameters_graphsage['num_epochs'], hyperparameters_graphsage['threshold']
    )

    visualize_train_history(
        hyperparameters_graphsage['num_epochs'], list_loss, list_valid_f1,
        f"GraphSAGE-{hyperparameters_graphsage['agg_type']}"
    )


def train_gat():
    model = GAT(
        node_features.shape[1],
        hyperparameters_gat['dim_out'], hyperparameters_gat['num_layers_nn'],
        hyperparameters_gat['dim_hidden_nn'], hyperparameters_gat['dropout'],
        hyperparameters_gat['alpha'], hyperparameters_gat['num_heads']
    ).to(device)

    list_loss, list_valid_f1, _ = train(
        model, node_features, edges_int, train_indices_msg, train_indices_pred,
        val_indices, test_indices, hyperparameters_gat['lr'], hyperparameters_gat['num_epochs'],
        hyperparameters_gat['threshold']
    )

    visualize_train_history(
        hyperparameters_gat['num_epochs'], list_loss, list_valid_f1, 'GAT'
    )

In [ ]:
for agg in ['gcn', 'mean', 'maxpool']:
    print("========================================================\n")
    train_graphsage(agg)
    print("\n========================================================")

In [ ]:
train_gat()

In [ ]:
ITER = 10

def get_mean_test_score(agg_type: str):
    logs = []
    for i in range(ITER):
        if agg_type != None:
            hyperparameters_graphsage['agg_type'] = agg_type
            model = GraphSageLinkPred(
            hyperparameters_graphsage['num_layers'],
            hyperparameters_graphsage['num_layers_nn'],
            hyperparameters_graphsage['dim_in'],
            hyperparameters_graphsage['dim_hidden'],
            hyperparameters_graphsage['dim_hidden_nn'],
            hyperparameters_graphsage['dim_out'],
            hyperparameters_graphsage['dropout'],
            hyperparameters_graphsage['agg_type']
            ).to(device)
        else:
            model = GAT(
                node_features.shape[1],
                hyperparameters_gat['dim_out'], hyperparameters_gat['num_layers_nn'],
                hyperparameters_gat['dim_hidden_nn'], hyperparameters_gat['dropout'],
                hyperparameters_gat['alpha'], hyperparameters_gat['num_heads']
            ).to(device)

        list_loss, list_valid_f1, test_f1 = train(
            model, node_features, edges_int, train_indices_msg, train_indices_pred, val_indices, test_indices,
            hyperparameters_graphsage['lr'], hyperparameters_graphsage['num_epochs'], hyperparameters_graphsage['threshold']
        )

        logs.append(test_f1.cpu().item())

    return np.array(logs).mean()

In [ ]:
score = get_mean_test_score('gcn')
print(f"Mean Test F1-score of GraphSAGE-GCN: {score}")

In [ ]:
score = get_mean_test_score('mean')
print(f"Mean Test F1-score of GraphSAGE-mean: {score}")

In [ ]:
score = get_mean_test_score('maxpool')
print(f"Mean Test F1-score of GraphSAGE-maxpool: {score}")

In [ ]:
score = get_mean_test_score(None)
print(f"Mean Test F1-score of GAT: {score}")